In [1]:
'''
Empirical analysis for DecisionTreeClassifier
    1. splitter
    2. max_depth
    3. presort
    4. max_features


'''

'\nEmpirical analysis for DecisionTreeClassifier\n    1. splitter\n    2. max_depth\n    3. presort\n    4. max_features\n\n\n'

In [2]:
from __future__ import division
from mnist import MNIST
from sklearn import tree
import numpy as np
from sklearn.metrics import classification_report,log_loss,accuracy_score,roc_auc_score
from time import time
import pandas as pd
from datetime import datetime
from openpyxl import load_workbook
import re
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix

In [3]:
from sklearn import tree

In [4]:
# Loading the data
mndata = MNIST('./')
images_train, labels_train = mndata.load_training()
images_test, labels_test = mndata.load_testing()
labels_test = np.array(labels_test)

images_train = np.array(images_train)
labels_train = np.array(labels_train)

In [5]:
def get_metrics(train_time,test_time,predictions,labels_test,images_test,clf):
    # Metrics calculation 
    accuracy = accuracy_score(predictions,labels_test,normalize=False)
    normalized_accuracy = accuracy_score(predictions,labels_test,normalize=True)
    log_loss_prediction = clf.predict_proba(images_test)
    log_loss_val = log_loss(labels_test,log_loss_prediction)     
    classfication_repo = classification_report(labels_test, predictions)
    
    labels_train_one_hot=label_binarize(labels_test,classes=[0,1,2,3,4,5,6,7,8,9])
    predictions_one_hot=label_binarize(predictions,classes=[0,1,2,3,4,5,6,7,8,9])
    micro_auc=roc_auc_score(labels_train_one_hot,predictions_one_hot,average="micro")
    macro_auc=roc_auc_score(labels_train_one_hot,predictions_one_hot,average="macro")
    
    cm = confusion_matrix(labels_test,predictions)
    cm_df = pd.DataFrame(data=cm)
    
    return [accuracy,normalized_accuracy,train_time,test_time,log_loss_val,macro_auc,micro_auc],cm_df,classfication_repo


In [6]:
def classifaction_report_to_dataframe(report):
    report_data = []
    lines = report.split('\n')
    for line in lines[2:-5]:
        row = {}
        row_data = line.split('      ')
        if len(row_data)>4:
            row['class'] = row_data[1]
            row['precision'] = float(row_data[2])
            row['recall'] = float(row_data[3])
            row['f1_score'] = float(row_data[4])
            row['support'] = float(row_data[5])
            report_data.append(row)
    for line in lines[-4:-1]:
        row = {}
        p = re.compile("[a-z]+ [a-z]+")
        m = p.search(line)  
        row['class'] = m.group(0)
        p = re.compile("\d+.?\d+")
        row_data = p.findall(line)
        row['precision'] = float(row_data[0])
        row['recall'] = float(row_data[1])
        row['f1_score'] = float(row_data[2])
        row['support'] = float(row_data[3])
        report_data.append(row)
        
        
    dataframe = pd.DataFrame.from_dict(report_data)
    return dataframe


In [7]:
excel_file_name = 'DecisionTree' + str(datetime.now().strftime('%Y%m%d%H%M%S')) + '.xlsx'
excel_writer = pd.ExcelWriter(excel_file_name, engine='openpyxl')
df = pd.DataFrame(columns=['Test'])
df.to_excel(excel_writer, sheet_name='demo_sheet')

excel_writer.save()

/usr/lib/python3/dist-packages/pandas/io/excel.py:687: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  self.book.remove_sheet(self.book.worksheets[0])


In [8]:
presort = ['best','random']

df = pd.DataFrame(columns=['Splitter', 'Accuracy','Normalized Accuracy', 
                 'Time_taken_train','Time_taken_test',
                 'Negative Log loss','Macro_auc','Micro_auc'])

for s in splitters:
    
    clf = tree.DecisionTreeClassifier(splitter=s)
    
    start = time()
    clf.fit(images_train, labels_train)
    train_time = time() - start
    
    start = time()
    predictions = clf.predict(images_test)
    test_time = time() - start
    
    metrics_array, cm_df,classfication_repo = get_metrics(train_time,test_time,predictions,labels_test,images_test,clf)
    
    df_data = [s] + metrics_array
    df.loc[len(df)] = df_data
    
    book = load_workbook(excel_file_name)
    with pd.ExcelWriter(excel_file_name, engine='openpyxl') as excel_writer:
        excel_writer.book = book
        df_classification_report = classifaction_report_to_dataframe(classfication_repo)
        df_classification_report.to_excel(excel_writer, sheet_name='splitters_cr' + '_'+ str(s))
        cm_df.to_excel(excel_writer, sheet_name='splitters_cr' + '_'+ str(s),startrow=len(df_classification_report)+5)
        excel_writer.save()

    
book = load_workbook(excel_file_name)
with pd.ExcelWriter(excel_file_name, engine='openpyxl') as excel_writer:
    excel_writer.book = book
    df.to_excel(excel_writer, sheet_name='splitters')
    excel_writer.save()

NameError: name 'splitters' is not defined

In [ ]:
max_depth_nodes = [1,2,5,10,15,20,25,30,35,40,None]
# max_depth_nodes = [1]
df = pd.DataFrame(columns=['Max depth', 'Accuracy','Normalized Accuracy', 
                 'Time_taken_train','Time_taken_test',
                 'Negative Log loss','Macro_auc','Micro_auc'])

for d in max_depth_nodes:
    
    clf = tree.DecisionTreeClassifier(max_depth=d)
    
    start = time()
    clf.fit(images_train, labels_train)
    train_time = time() - start
    
    start = time()
    predictions = clf.predict(images_test)
    test_time = time() - start
    
    metrics_array, cm_df, classfication_repo = get_metrics(train_time,test_time,predictions,labels_test,images_test,clf)
    
    df_data = [d] + metrics_array
    df.loc[len(df)] = df_data
    
    book = load_workbook(excel_file_name)
    with pd.ExcelWriter(excel_file_name, engine='openpyxl') as excel_writer:
        excel_writer.book = book
        df_classification_report = classifaction_report_to_dataframe(classfication_repo)
        df_classification_report.to_excel(excel_writer, sheet_name='max_depth_cr' + '_'+ str(d))
        cm_df.to_excel(excel_writer, sheet_name='max_depth_cr' + '_'+ str(d),startrow=len(df_classification_report)+5)
        excel_writer.save()

    
book = load_workbook(excel_file_name)
with pd.ExcelWriter(excel_file_name, engine='openpyxl') as excel_writer:
    excel_writer.book = book
    df.to_excel(excel_writer, sheet_name='max_depth')
    excel_writer.save()

In [ ]:
presort_features = [False,True]
# presort_features = [False]
df = pd.DataFrame(columns=['presort', 'Accuracy','Normalized Accuracy', 
                 'Time_taken_train','Time_taken_test',
                 'Negative Log loss','Macro_auc','Micro_auc'])

for d in presort_features:
    
    clf = tree.DecisionTreeClassifier(presort=d)
    
    start = time()
    clf.fit(images_train, labels_train)
    train_time = time() - start
    
    start = time()
    predictions = clf.predict(images_test)
    test_time = time() - start
    
    metrics_array, cm_df,classfication_repo = get_metrics(train_time,test_time,predictions,labels_test,images_test,clf)
    
    df_data = [d] + metrics_array
    df.loc[len(df)] = df_data
    
    book = load_workbook(excel_file_name)
    with pd.ExcelWriter(excel_file_name, engine='openpyxl') as excel_writer:
        excel_writer.book = book
        df_classification_report = classifaction_report_to_dataframe(classfication_repo)
        df_classification_report.to_excel(excel_writer, sheet_name='presort_cr' + '_'+ str(d))
        cm_df.to_excel(excel_writer, sheet_name='presort_cr' + '_'+ str(d),startrow=len(df_classification_report)+5)
        excel_writer.save()

    
book = load_workbook(excel_file_name)
with pd.ExcelWriter(excel_file_name, engine='openpyxl') as excel_writer:
    excel_writer.book = book
    df.to_excel(excel_writer, sheet_name='presort')
    excel_writer.save()

In [ ]:
max_features = [1,2,5,10,15,20,25,30,50,100,200,300,400]
# max_features = [1]
df = pd.DataFrame(columns=['Max features', 'Accuracy','Normalized Accuracy', 
                 'Time_taken_train','Time_taken_test',
                 'Negative Log loss','Macro_auc','Micro_auc'])

for d in max_features:
    
    clf = tree.DecisionTreeClassifier(max_features=d)
    
    start = time()
    clf.fit(images_train, labels_train)
    train_time = time() - start
    
    start = time()
    predictions = clf.predict(images_test)
    test_time = time() - start
    
    metrics_array, cm_df, classfication_repo = get_metrics(train_time,test_time,predictions,labels_test,images_test,clf)
    
    df_data = [d] + metrics_array
    df.loc[len(df)] = df_data
    
    book = load_workbook(excel_file_name)
    with pd.ExcelWriter(excel_file_name, engine='openpyxl') as excel_writer:
        excel_writer.book = book
        df_classification_report = classifaction_report_to_dataframe(classfication_repo)
        df_classification_report.to_excel(excel_writer, sheet_name='max_features_cr' + '_'+ str(d))
        cm_df.to_excel(excel_writer, sheet_name='max_features_cr' + '_'+ str(d),startrow=len(df_classification_report)+5)
        excel_writer.save()

    
book = load_workbook(excel_file_name)
with pd.ExcelWriter(excel_file_name, engine='openpyxl') as excel_writer:
    excel_writer.book = book
    df.to_excel(excel_writer, sheet_name='max_features')
    excel_writer.save()